In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
df_uni = pd.read_pickle("../notebooks/universidades.pkl")

df_uni.head(1)

,web_pages,alpha_two_code,state_province,name,country,lat,long
0,http://www.atlantida.edu.ar/,AR,Buenos Aires,Universidad Atlantida Argentina,Argentina,-34.607568,-58.437089


In [3]:
class Cargar:

    """Esta clase va a recibir dos parámetros, que son el nombre de la base de datos que se quiere crear
    y la contraseña para poder crearla"""
    
    def __init__(self, nombre_bbdd, contraseña):

        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        """Esta función recibe los parámetros del método constructor. Crea la base de datos si no existía
        con anterioridad y, si no, devuelve un mensaje de que no se ha podido crear."""

        mydb = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=f'{self.contraseña}') 
        mycursor = mydb.cursor()

        print("La conexión se ha realizado con éxito.")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")


            print(f"La base de datos {self.nombre_bbdd} se ha creado con éxito o ya existía anteriormente.")
            
        except:

            print(f"La base de datos {self.nombre_bbdd} no se ha podido crear.")


    def crear_tablas(self):

        """Esta función recibe los parámetros del método constructor. Crea las tablas de
        la base de datos si no existían con anterioridad y, si no, devuelve un mensaje avisando de que
        no se ha podido crear."""


        mydb = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=f'{self.contraseña}') 
        mycursor = mydb.cursor()


        query1 = f"""CREATE TABLE IF NOT EXISTS `{self.nombre_bbdd}`.`países` (
                    `idestado` INT NOT NULL AUTO_INCREMENT,
                    `nombre_país` VARCHAR(45) NOT NULL,
                    `nombre_provincia` VARCHAR(45) NOT NULL,
                    `latitud` DECIMAL (10, 7) NULL,
                    `longitud` DECIMAL (10, 7) NULL,
                    PRIMARY KEY (`idestado`));"""
        
        query2 = f"""CREATE TABLE IF NOT EXISTS `{self.nombre_bbdd}`.`universidades` (
                    `id_universidad` INT NOT NULL AUTO_INCREMENT,
                    `nombre_universidad` VARCHAR(100) NULL,
                    `pagina_web` VARCHAR(100) NULL,
                    `países_idestado` INT NOT NULL,
                    PRIMARY KEY (`id_universidad`),
                    INDEX `fk_universidades_países_idx` (`países_idestado` ASC) VISIBLE,
                    CONSTRAINT `fk_universidades_países`
                        FOREIGN KEY (`países_idestado`)
                        REFERENCES `{self.nombre_bbdd}`.`países` (`idestado`)
                        ON DELETE NO ACTION
                        ON UPDATE NO ACTION); """   

        try:
            
            mycursor.execute(query1)
            mycursor.execute(query2)
            mydb.commit()
            print("Las tablas de la base de datos se han podido crear correctamente.")

        except mysql.connector.Error as err:

            print("No se han podido crear las tablas de la base de datos.")
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg) 

    def cargar_provincias(self, dataframe):

        """Esta función recibe como parámetro un DataFrame para cargar los datos correspondientes de país, 
        estado, latitud y longitud a la tabla de "países" de la base de datos creada. 
        Primero comprueba que no esté ya insertada en la base de datos y, si no lo está, 
        la inserta; de lo contrario, avisa al usuario de que ya está ese registro en la 
        tabla"""

        df_provincias = dataframe[["country", "state_province", "lat", "long"]].drop_duplicates()
        # Creamos un DataFrame con las columnas que nos interesan del DataFrame 
        # especificado al llamar a la función, y quitamos los duplicados por si acaso.

        mydb = mysql.connector.connect(user="root",
                                    password= f"{self.contraseña}",
                                    host="localhost", 
                                    database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
    
        for indice, fila in df_provincias.iterrows(): # Iteramos por cada una de las filas del DataFrame creado

            try: # Chequeamos que no haya ningún registro que coincida en estado y país

                check_provincia = f"""SELECT idestado FROM países 
                                    WHERE nombre_provincia = "{fila['state_province']}" AND nombre_país = "{fila['country']}";"""
                mycursor.execute(check_provincia)
                existe_provincia = mycursor.fetchone()
                

                if not existe_provincia: #Si no existe un registro, lo inserta

                    query_provincia = f"""INSERT INTO países (nombre_país, nombre_provincia, latitud, longitud) 
                                            VALUES ("{fila['country']}", "{fila['state_province']}", 
                                            {'NULL' if pd.isna(fila['lat']) else fila['lat']},
                                            {'NULL' if pd.isna(fila['long']) else fila['long']}); 
                                            """
                                            # Hay algunos nulos, así que indicamos que ponga NULL en la base de datos
                                            #para que no haya errores. 
                    
                    mycursor.execute(query_provincia)
                    mydb.commit()

                else: # Si existe, devuelve que ya existe.
                    print(f"Ya existe la provincia {fila['state_province']} en la base de datos.")

            
            except mysql.connector.Error as err: #Si hay un error, devuelve el código y el mensaje.

                print("No se han podido cargar las provincias a la base de datos.")
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)          


    def cargar_universidades(self, dataframe):

        """Esta función recibe como parámetro un DataFrame para cargar los datos correspondientes de nombre de 
        la universidad, su página web, el país y el estado a la tabla de "universidades" de la base de datos creada. 
        Primero comprueba que no esté ya insertada en la base de datos y, si no lo está, la inserta; de lo contrario, avisa al usuario de que ya está ese registro en la 
        tabla. Además, comprueba el id correspondiente a la ciudad y país, para añadirlo a la tabla "universidades"
        como *foreing key*"""


        df_universidad = dataframe[["name", "web_pages", "country", "state_province"]].drop_duplicates()
                # Creamos un DataFrame con las columnas que nos interesan del DataFrame 
                # especificado al llamar a la función, y quitamos los duplicados por si acaso.


        mydb = mysql.connector.connect(user="root",
                                    password= f"{self.contraseña}",
                                    host="localhost", 
                                    database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        for indice, fila in df_universidad.iterrows(): # Iteramos por cada una de las filas del DataFrame creado

            try: # Chequeamos que no haya ningún registro que coincida en nombre de universidad

                check_universidad = f"""SELECT id_universidad FROM universidades 
                                        WHERE nombre_universidad = "{fila['name']}"; 
                                        """
                mycursor.execute(check_universidad)
                existe_universidad = mycursor.fetchone()
                    
                if not existe_universidad: #Si no existe un registro, continua

                    try: # Obtiene el ID del estado correspondiente en la tabla de "paises" y lo inserta
                        # como foreign key con el resto de datos de nombre y página web.

                        check_provincia = f"""SELECT idestado FROM países 
                                            WHERE nombre_provincia = "{fila['state_province']}" AND nombre_país = "{fila['country']}";"""
                                            
                        mycursor.execute(check_provincia)

                        id_estado = mycursor.fetchone()[0]
        
                        query_universidad = f"""INSERT INTO universidades (nombre_universidad, pagina_web, países_idestado) 
                                                VALUES ("{fila['name']}", "{fila['web_pages']}", "{id_estado}");
                                                """
                                
                        mycursor.execute(query_universidad)
                        mydb.commit() 
                    
                    except mysql.connector.Error as err: # Si no puede obtener el ID del estado
                                                            #devuelve los errores

                        print("No se ha podido  obtener el ID de la provincia.")
                        print(err)
                        print("Error Code:", err.errno)
                        print("SQLSTATE", err.sqlstate)
                        print("Message", err.msg)     

                else: # Si sí existe la universidad en esa tabla, devuelve que ya existe.

                    print(f"Ya existe la universidad {fila['name']} en la base de datos.")

                
            except mysql.connector.Error as err:

                print("No se han podido cargar las universidades a la base de datos.")
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)       


In [5]:
bbdd = Cargar("universidades", "AlumnaAdalab")

In [6]:
bbdd.crear_bbdd()

La conexión se ha realizado con éxito.
La base de datos universidades se ha creado con éxito o ya existía anteriormente.


In [7]:
bbdd.crear_tablas()

Las tablas de la base de datos se han podido crear correctamente.


In [9]:
bbdd.cargar_provincias(df_uni)

In [11]:
bbdd.cargar_universidades(df_uni)

Ya existe la universidad Gateway Community College en la base de datos.
